# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 2

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        # 'NYC_summer_2010_99p',
        # 'NYC_summer_2015_50p',
        # 'NYC_summer_2017_25p',
        # 'NYC_summer_2018_75p'
    ]),
    ('PHX_Heat_Test', [
        'PHX_summer_2008_25p',
        # 'PHX_summer_2009_50p',
        # 'PHX_summer_2011_99p',
        # 'PHX_summer_2015_75p',
        # 'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

print(sim_names)
client = storage.Client(project="climateiq")


2025-01-21 17:07:33.968282: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 17:07:34.016792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 17:07:34.016823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 17:07:34.018018: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 17:07:34.025353: I tensorflow/core/platform/cpu_feature_guar

['NYC_Heat_Test/NYC_summer_2000_01p', 'PHX_Heat_Test/PHX_summer_2008_25p']


In [2]:
train_frac = 0.8

# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_fake_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
).batch(batch_size=batch_size)

# Create validation dataset with fused spatiotemporal data
val_ds = dataset.load_fake_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
).batch(batch_size=batch_size)


INFO:root:Total simulation days before filtering: 200
2025-01-21 17:07:45.212826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 886 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0
INFO:root:Total simulation days before filtering: 200


In [ ]:
train_frac = 0.8

# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(0.0, train_frac),
).batch(batch_size=batch_size)

# Create validation dataset with fused spatiotemporal data
val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    hash_range=(train_frac, 1.0),
).batch(batch_size=batch_size)


In [ ]:
num_samples = 0
for batch in train_ds:
    num_samples += batch[0]['spatiotemporal'].shape[0]
print("Number of samples:", num_samples)

In [ ]:
num_samples = 0
for batch in val_ds:
    num_samples += batch[0]['spatiotemporal'].shape[0]
print("Number of samples:", num_samples)

In [4]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)

In [ ]:
# Create a unique log directory by appending the current timestamp
log_dir = os.path.join("./logs", "run_" + time.strftime("%Y%m%d-%H%M%S"))

# Set up TensorBoard callback
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
print(log_dir)

In [ ]:
# Train the model
model.fit( train_ds, val_ds, epochs=500, callbacks=[tb_callback])



In [ ]:
model._model.save("saved_model/atmo_model2", save_format="tf")


In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("saved_model/atmo_model")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions from the validation set
predictions = model._model.predict(val_ds)  # Use the underlying Keras model

# Assuming the structure of val_ds returns (input_data, ground_truth)
for input_data, ground_truth in val_ds.take(1):  # Taking just one batch from val_ds
    # Get predicted labels
    predicted_labels = model._model.predict(input_data)
    
    # Visualize the first sample
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Ground Truth Visualization
    axes[0].imshow(ground_truth[0,:, :, :, 0], cmap='viridis')  # Adjust if output shape differs
    axes[0].set_title('Ground Truth')

    # Prediction Visualization
    axes[1].imshow(predicted_labels[0, :,:, :, 0], cmap='viridis')  # Adjust if output shape differs
    axes[1].set_title('Predicted Labels')

    plt.show()
    break  # Break after visualizing one batch


In [ ]:
predicted_labels.shape
ground_truth[0, :, :, :].shape
predicted_labels[0, :, :, :].shape
ground_truth[0, 0, :, :, channel_index].shape

In [ ]:
# Select the first image in the batch
sample_index = 14  # Index of the sample in the batch
channel_index = 1  # Select the first channel to visualize

# Extract data for visualization
ground_truth_image = ground_truth[sample_index, 0, :, :, channel_index]  # Shape: (200, 200)
predicted_image = predicted_labels[sample_index,0,  :, :, channel_index]  # Shape: (200, 200)

# Visualize the ground truth and predictions
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Ground Truth Visualization
axes[0].imshow(ground_truth_image, cmap='viridis')
axes[0].set_title('Ground Truth (Channel 0)')

# Prediction Visualization
axes[1].imshow(predicted_image, cmap='viridis')
axes[1].set_title('Predicted Labels (Channel 0)')

plt.show()

In [ ]:
predictions = model._model.predict(val_ds)  # Use the underlying Keras model
# Select the first image in the batch
sample_index = 0  # Index of the sample in the batch
channel_index = 1  # Select the first channel to visualize
for input_data, ground_truth in val_ds.take(1):  # Taking just one batch from val_ds
    # Extract data for visualization
    predicted_labels = model._model.predict(input_data)
    ground_truth_image = ground_truth[sample_index, 0, :, :, channel_index]  # Shape: (200, 200)
    predicted_image = predicted_labels[sample_index,0,  :, :, channel_index]  # Shape: (200, 200)

    # Visualize the ground truth and predictions
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Ground Truth Visualization
    axes[0].imshow(ground_truth_image, cmap='viridis')
    axes[0].set_title('Ground Truth (Channel 0)')

    # Prediction Visualization
    axes[1].imshow(predicted_image, cmap='viridis')
    axes[1].set_title('Predicted Labels (Channel 0)')

    plt.show()

In [ ]:
for batch in train_ds.take(1):
    print(type(batch))  # Check the type
    #print(batch)        # Print batch content to inspect its structure
    break


In [ ]:
input_data['spatiotemporal'].shape[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract a single batch from the training dataset
for input_data, ground_truth in train_ds.take(1):  # Take one batch
    # Dynamically fetch batch size and channels
    batch_size = 25  # Number of samples in the batch
    num_channels_input = 34 # Input channels
    num_channels_ground_truth = 5  # Ground truth channels

    
    # Number of samples to visualize (adjust based on batch size)
    num_samples_to_plot = min(4, batch_size)  # Plot at most 4 samples

    # Plot input data and corresponding ground truth
    for i in range(num_samples_to_plot):
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        # Input Data Visualization (Select Channel 0 for visualization)
        axes[0].imshow(input_data['spatiotemporal'][i, :, :, 2], cmap='viridis')  # Adjust channel index if needed
        axes[0].set_title(f"Input Data - Sample {i+1}")

        # Ground Truth Visualization (Select Channel 0 for visualization)
        axes[1].imshow(ground_truth[i,0, :, :, 0], cmap='viridis')  # Adjust channel index if needed
        axes[1].set_title(f"Ground Truth - Sample {i+1}")

        plt.tight_layout()
        plt.show()

    break  # Exit after visualizing one batch


In [ ]:
inputs, labels = next(iter(train_ds))
{key: tensor.shape for key, tensor in inputs.items()}

In [ ]:
model._model.summary()

In [ ]:
# Test calling the model on some training data
inputs, labels = next(iter(train_ds))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)